In [0]:
%pylab inline

In [0]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [0]:
# Example: load a DSS dataset as a Pandas dataframe
mydataset = dataiku.Dataset("mydataset")
mydataset_df = mydataset.get_dataframe()

In [0]:
import dataiku

# client initialization:
client = dataiku.api_client().get_connection("postgresql")
conn.get_info()["params"]['password']

In [0]:
conn = dataiku.api_client().get_connection("postgresql")
conn.get_location_info()["params"]["password"]

In [10]:
import dataiku
import datetime

# client initialization:
client = dataiku.api_client()

project_key = "CUSTOMERSCASES"
project = client.get_project(project_key)
project_info = project.get_timeline()
print(project_info["items"][0])

{'user': 'admin', 'action': 'JUPYTER_NOTEBOOK_EDIT', 'projectKey': 'CUSTOMERSCASES', 'objectType': 'JUPYTER_NOTEBOOK', 'objectId': 'py36 notebook', 'details': {'objectDisplayName': 'py36 notebook', 'userDisplayName': 'Administrator'}, 'time': 1672817656126}
